# Vergleich von Imputation Methoden

An dieser Stelle sollen verschiede Methoden zum interpolieren von fehlenden Werten betrachtet und verglichen werden.

### Vorbereitung

In [1]:
import pandas as pd
import numpy as np
import math
import time

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer

t0 = time.time()

In [2]:
results = []

In [3]:
def reset_base():
    base= pd.read_csv('additional_data/base.csv') 
    base.set_index(['Country Name', 'Indicator Name'], inplace=True)
    base = base.sort_index(level=['Country Name', 'Indicator Name'])
    return base
base = reset_base()

def get_cords(frac, rnd_state):
    n = int(base.isna().sum().sum()*frac)
    print(f'Testdaten mit {frac*100}% fehlenden Werten (absolut: {n})')
    #random state to ensure reproducibility
    rnds = np.random.RandomState(rnd_state)

    #coordinates for data entries to be removed randomly
    #5000 entries are selected
    cords = pd.DataFrame([[rnds.randint(0, len(base), size=n*4)[i], 
                  rnds.randint(0, len(base.columns), size=n*4)[i]]
                  for i in range(n*4)])

    #all coordinates pointing to NaN entries are removed and
    #first 1000 remaining entries are selected
    cords['value'] = [base.iloc[cords[0][i], cords[1][i]] for i in cords.index]
    cords = cords.dropna()[:n].reset_index(drop=True)
    
    return cords

def reset_train(cords):
    train = reset_base()
    for i in cords.index:
        train.iloc[cords[0][i], cords[1][i]] = None
    return train

def evaluate(df, t, cords):
    
    
    #scaling original data and imputed data
    #necessary ?????????????????????????????????????
    train = reset_train(cords)
    scaler = StandardScaler().fit(train) #fitting on train?
    norm_base = pd.DataFrame(scaler.transform(base))
    df = pd.DataFrame(scaler.transform(df))

    #getting imputed values for simulated NaNs and true value 
    res =pd.DataFrame({'y_true': [norm_base.iloc[cords[0][i], cords[1][i]] for i in cords.index],
                       'y_pred': [df.iloc[cords[0][i], cords[1][i]] for i in cords.index]
                      })
    res = res.dropna()

   
    #calculate evaluation metrics
    r2 = r2_score(res['y_true'], res['y_pred'])
    rmse = math.sqrt(mean_squared_error(res['y_true'], res['y_pred']))
    still_missing = df.isna().sum().sum()
    

    print(f'r2: {r2}, rmse: {rmse}, t: {t}')
    print('')
    
    return [r2, rmse, still_missing, t]



In [4]:
def impute_backfill(df):
    df = df.fillna(method='bfill', limit=3)
    return df

def impute_overall_means(df):
    #fill NaNs with overall mean of that indicator
    values = pd.DataFrame(df.stack()).groupby('Indicator Name')[0].mean()
    df = pd.DataFrame(df.stack(dropna=False))
    
    df[0] = df[0].fillna(df.groupby('Indicator Name')[0].transform('mean'))
    df = df.unstack()
    df.columns = df.columns.droplevel(0)
    df = df.sort_index(level=['Country Name', 'Indicator Name'])
        
    return df

def impute_yearly_means(df):
    #fill NaNs with overall mean of that indicator
    
    for i in df.columns:
        df[i] = df[i].fillna(df.groupby('Indicator Name')[i].transform('mean'))
            
    return df

def impute_yearly_means_per_region(df):
    country_data = pd.read_csv('../Data/WDICountry.csv')
    country_data = country_data.loc[:,['Table Name', 'Region']]
    df = pd.merge(df.reset_index(), country_data, how='left', left_on='Country Name', right_on='Table Name').drop('Table Name', axis=1)
    df = df.set_index(['Country Name', 'Indicator Name', 'Region'])

    for i in df.columns:
        df[i] = df[i].fillna(df.groupby(['Indicator Name', 'Region'])[i].transform('mean'))

    df = df.reset_index().set_index(['Country Name', 'Indicator Name']).drop('Region', axis=1)
    return df

def interpolate3(df):
    df = df.interpolate(limit=3)
    return df

def interpolate_all(df):
    df = df.interpolate()
    return df

def iterative_imputer_1(df):
    col = df.columns
    idx = df.index
    
    iter_imp = IterativeImputer(random_state=999)
    df= iter_imp.fit_transform(df)
    df= pd.DataFrame(df, columns=col, index=idx)
    return df

def iterative_imputer_2(df):
    df = df.unstack().T
    col = df.columns
    idx = df.index

    iter_imp = IterativeImputer(random_state=999)
    df= iter_imp.fit_transform(df)

    df = pd.DataFrame(df, columns=col, index=idx)
    df = df.unstack().T
    df = df.sort_index(level=['Country Name', 'Indicator Name'])
    
    return df

def iterative_imputer_3(df):

    df = df.reset_index()
    df = df.set_index(['Indicator Name', 'Country Name'])
    df = df.unstack().T

    col = df.columns
    idx = df.index

    iter_imp = IterativeImputer(random_state=999, verbose=True)
    df= iter_imp.fit_transform(df)

    df = pd.DataFrame(df, columns=col, index=idx)
    df = df.unstack().T
    df = df.reset_index()
    df = df.set_index(['Country Name', 'Indicator Name'])
    df = df.sort_index(level=['Country Name', 'Indicator Name'])
    
    return df

def mice_imputer(df):
    n_imputations =  12
    dfs = []
    col = df.columns
    idx = df.index
    
    for i in range(n_imputations): 
        print(f'Imputation round {i}')
        iter_imp = IterativeImputer(random_state=i, sample_posterior=True, verbose=2)
        df_temp = iter_imp.fit_transform(df)
        dfs.append(df_temp)
    
    df = np.mean(np.array(dfs), axis=0)
    df = pd.DataFrame(df, columns=col, index=idx)
    return df

def knn_imputer1(df):
    col = df.columns
    idx = df.index
    
    knn_imp = KNNImputer(n_neighbors=2)
    df= knn_imp.fit_transform(df)
    df = pd.DataFrame(df, columns=col, index=idx)
    return df

def knn_imputer2(df, n=4):
    
    df = df.reset_index()
    df = df.set_index(['Indicator Name', 'Country Name'])
    df = df.unstack().T

    col = df.columns
    idx = df.index

    knn_imp = KNNImputer(n_neighbors=n)
    df= knn_imp.fit_transform(df)
    df = pd.DataFrame(df, columns=col, index=idx)

    df = df.unstack().T
    df = df.reset_index()
    df = df.set_index(['Country Name', 'Indicator Name'])
    df = df.sort_index(level=['Country Name', 'Indicator Name'])
    
    return df

In [5]:
functions =  {'Backfill':impute_backfill, 'Overall Mean':impute_overall_means, 
              'Yearly Mean':impute_yearly_means, 'Yearly Mean per Region':impute_yearly_means_per_region, 
              'Interpolate 3':interpolate3, 'Interpolate all':interpolate_all, 
              'ICE 1':iterative_imputer_1, 'ICE 2':iterative_imputer_2, 
              'ICE 3':iterative_imputer_3, 'MICE':mice_imputer, 
              'KNN 1':knn_imputer1, 'KNN 2':knn_imputer2
             }

In [6]:
def run_all(frac):
    
    cords_list =[]
    n = 7
    for i in np.arange(1,n+1):
        cords_list.append(get_cords(frac, i))
    
    results = {}
    detailed_results = {}    
    
    for name, func in functions.items():
        func_runs = []
        print(name)
        
        for count, cords in enumerate(cords_list):
            
            print(f'run: {count+1}/{n}')
            t0 = time.time()
            df= func(reset_train(cords)) 
            t1 = time.time()

            t = t1-t0

            func_runs.append(evaluate(df, t, cords))
        
        print(func_runs)
        print ('')
        detailed_results[name] = func_runs
        results[name] = np.mean(np.array(func_runs), axis=0)
        
    return results, detailed_results


In [7]:
r1, r1_detail = run_all(0.05)

Testdaten mit 5.0% fehlenden Werten (absolut: 8515)
Testdaten mit 5.0% fehlenden Werten (absolut: 8515)
Testdaten mit 5.0% fehlenden Werten (absolut: 8515)
Testdaten mit 5.0% fehlenden Werten (absolut: 8515)
Testdaten mit 5.0% fehlenden Werten (absolut: 8515)
Testdaten mit 5.0% fehlenden Werten (absolut: 8515)
Testdaten mit 5.0% fehlenden Werten (absolut: 8515)
Backfill
run: 1/7
r2: -0.2630386067986965, rmse: 3.1264580984032304, t: 0.8915562629699707

run: 2/7
r2: -2.77533240911633, rmse: 1.1896157293266358, t: 0.8860158920288086

run: 3/7
r2: -1.688280205196218, rmse: 1.0226797653748734, t: 0.8775792121887207

run: 4/7
r2: -0.6672076075877178, rmse: 2.085863590967128, t: 0.8720483779907227

run: 5/7
r2: -0.07720522781720884, rmse: 3.533882957257935, t: 0.8745901584625244

run: 6/7
r2: -5.6191599013569355, rmse: 1.7468428288779958, t: 0.8996379375457764

run: 7/7
r2: -3.5433278053623427, rmse: 1.259265501149907, t: 0.8805661201477051

[[-0.2630386067986965, 3.1264580984032304, 32924, 0

C:\Users\schwa\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


r2: 0.9942158016960598, rmse: 0.20756410705991854, t: 9.991528511047363

run: 2/7


C:\Users\schwa\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


r2: 0.7827329807258437, rmse: 0.2820413273792655, t: 9.93337607383728

run: 3/7


...

[IterativeImputer] Ending imputation round 8/10, elapsed time 9.22
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.34
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.50
Imputation round 7
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.12
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.28
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.41
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.59
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.73
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.87
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.00
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.15
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.27
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.44
Imputation round 8
[IterativeImputer] Comple

[IterativeImputer] Ending imputation round 6/10, elapsed time 6.92
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.08
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.24
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.40
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.54
Imputation round 6
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.20
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.33
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.51
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.64
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.82
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.96
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.15
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.30
[IterativeImputer] Ending imputation round 9/10

[IterativeImputer] Ending imputation round 4/10, elapsed time 4.56
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.73
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.89
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.06
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.20
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.38
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.50
Imputation round 5
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.13
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.27
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.43
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.58
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.74
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.87
[IterativeImputer] Ending imputation round 7/10

[IterativeImputer] Ending imputation round 2/10, elapsed time 2.30
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.46
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.60
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.76
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.90
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.05
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.20
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.38
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.52
Imputation round 4
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.16
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.30
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.47
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.61
[IterativeImputer] Ending imputation round 5/10

r2: 0.3850929761290802, rmse: 2.140106546564469, t: 18.854260206222534

run: 2/7
r2: 0.7764619936100474, rmse: 0.28608265888433726, t: 18.92342257499695

run: 3/7
r2: 0.018198179415029214, rmse: 0.6086215962503441, t: 18.870671272277832

run: 4/7
r2: 0.03975169532798306, rmse: 1.5580936242262533, t: 18.97544527053833

run: 5/7
r2: 0.3651864550278935, rmse: 2.6687042009340893, t: 18.601709842681885

run: 6/7
r2: -0.009893226396108457, rmse: 0.6719345144571093, t: 18.870542526245117

run: 7/7
r2: 0.6508860975718154, rmse: 0.3434159398405326, t: 18.80156683921814

[[0.3850929761290802, 2.140106546564469, 0, 18.854260206222534], [0.7764619936100474, 0.28608265888433726, 0, 18.92342257499695], [0.018198179415029214, 0.6086215962503441, 0, 18.870671272277832], [0.03975169532798306, 1.5580936242262533, 0, 18.97544527053833], [0.3651864550278935, 2.6687042009340893, 0, 18.601709842681885], [-0.009893226396108457, 0.6719345144571093, 0, 18.870542526245117], [0.6508860975718154, 0.34341593984053

In [8]:
r1 = pd.DataFrame(r1).T
r1

,0,1,2,3
Backfill,-2.090507,1.994944,32841.000000,0.883142
Overall Mean,-0.375690,1.520351,0.000000,1.148196
Yearly Mean,0.003883,1.443118,52298.000000,0.975204
Yearly Mean per Region,-0.037073,1.459472,58352.714286,1.096290
Interpolate 3,-0.821449,1.713417,32871.428571,0.954637
Interpolate all,-0.821336,1.685872,58.000000,0.932401
ICE 1,0.958789,0.173133,0.000000,9.967214
ICE 2,0.936942,0.354860,0.000000,24.333753
ICE 3,0.744604,0.731466,0.000000,85.888058
MICE,0.961106,0.190552,0.000000,139.684988


In [9]:
r1_detail = pd.DataFrame(pd.DataFrame(r1_detail).T.stack().to_dict()).T
r1_detail.columns = ['r2', 'rmse', 'still_missing', 't']
r1_detail

r2      rmse  still_missing          t
Backfill 0 -0.263039  3.126458        32924.0   0.891556
         1 -2.775332  1.189616        32825.0   0.886016
         2 -1.688280  1.022680        32824.0   0.877579
         3 -0.667208  2.085864        32841.0   0.872048
         4 -0.077205  3.533883        32828.0   0.874590
...              ...       ...            ...        ...
KNN 2    2  0.018198  0.608622            0.0  18.870671
         3  0.039752  1.558094            0.0  18.975445
         4  0.365186  2.668704            0.0  18.601710
         5 -0.009893  0.671935            0.0  18.870543
         6  0.650886  0.343416            0.0  18.801567

[84 rows x 4 columns]

In [10]:
r2 , r2_detail =  run_all(0.075)

Testdaten mit 7.5% fehlenden Werten (absolut: 12773)
Testdaten mit 7.5% fehlenden Werten (absolut: 12773)
Testdaten mit 7.5% fehlenden Werten (absolut: 12773)
Testdaten mit 7.5% fehlenden Werten (absolut: 12773)
Testdaten mit 7.5% fehlenden Werten (absolut: 12773)
Testdaten mit 7.5% fehlenden Werten (absolut: 12773)
Testdaten mit 7.5% fehlenden Werten (absolut: 12773)
Backfill
run: 1/7
r2: -0.7622360204251646, rmse: 1.9101575370444133, t: 1.252185583114624

run: 2/7
r2: -0.13326755926364164, rmse: 1.5732663625326984, t: 1.2441728115081787

run: 3/7
r2: -0.15471382925960686, rmse: 2.34729184379117, t: 1.245006799697876

run: 4/7
r2: -0.38762500614250417, rmse: 2.1233606768359277, t: 1.2373647689819336

run: 5/7
r2: -0.12638032635757868, rmse: 1.8674634798834449, t: 1.2455017566680908

run: 6/7
r2: -0.1353819730423571, rmse: 1.8854985255726455, t: 1.239548683166504

run: 7/7
r2: -1.259193916505637, rmse: 1.778120493027196, t: 1.2486271858215332

[[-0.7622360204251646, 1.9101575370444133,

C:\Users\schwa\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


r2: 0.9803957096839419, rmse: 0.19838139998923512, t: 10.240909337997437

run: 2/7


C:\Users\schwa\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


r2: 0.9755494244776735, rmse: 0.22712199683626336, t: 10.305448293685913

run: 3/7


...

[IterativeImputer] Ending imputation round 8/10, elapsed time 10.06
[IterativeImputer] Ending imputation round 9/10, elapsed time 11.29
[IterativeImputer] Ending imputation round 10/10, elapsed time 12.48
Imputation round 7
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.24
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.45
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.69
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.91
[IterativeImputer] Ending imputation round 5/10, elapsed time 6.18
[IterativeImputer] Ending imputation round 6/10, elapsed time 7.41
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.67
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.90
[IterativeImputer] Ending imputation round 9/10, elapsed time 11.15
[IterativeImputer] Ending imputation round 10/10, elapsed time 12.38
Imputation round 8
[IterativeImputer] Compl

[IterativeImputer] Ending imputation round 6/10, elapsed time 6.94
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.08
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.25
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.39
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.55
Imputation round 6
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.16
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.32
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.44
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.63
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.78
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.95
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.10
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.28
[IterativeImputer] Ending imputation round 9/10

[IterativeImputer] Ending imputation round 4/10, elapsed time 4.61
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.75
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.93
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.05
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.22
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.36
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.52
Imputation round 5
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.14
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.30
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.44
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.59
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.73
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.90
[IterativeImputer] Ending imputation round 7/10

[IterativeImputer] Ending imputation round 2/10, elapsed time 2.32
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.48
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.62
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.79
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.90
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.06
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.19
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.37
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.51
Imputation round 4
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.16
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.31
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.48
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.62
[IterativeImputer] Ending imputation round 5/10

r2: -1.602264991548012, rmse: 2.2856048436310656, t: 19.485417366027832

run: 2/7
r2: 0.3021798150724092, rmse: 1.2133509054211025, t: 19.627933025360107

run: 3/7
r2: 0.46955096762140014, rmse: 1.5636010429853207, t: 19.485605001449585

run: 4/7
r2: 0.3733797406161049, rmse: 1.4021077287819956, t: 19.581395387649536

run: 5/7
r2: 0.12532994575125067, rmse: 1.619596599110858, t: 19.49038553237915

run: 6/7
r2: 0.08000077527047134, rmse: 1.6680486031875468, t: 19.675562143325806

run: 7/7
r2: 0.2418860703417669, rmse: 1.0132411753116484, t: 19.355597972869873

[[-1.602264991548012, 2.2856048436310656, 0, 19.485417366027832], [0.3021798150724092, 1.2133509054211025, 0, 19.627933025360107], [0.46955096762140014, 1.5636010429853207, 0, 19.485605001449585], [0.3733797406161049, 1.4021077287819956, 0, 19.581395387649536], [0.12532994575125067, 1.619596599110858, 0, 19.49038553237915], [0.08000077527047134, 1.6680486031875468, 0, 19.675562143325806], [0.2418860703417669, 1.0132411753116484, 0

In [11]:
r2 = pd.DataFrame(r2).T
r2

,0,1,2,3
Backfill,-0.422686,1.926451,33270.142857,1.244630
Overall Mean,-0.073224,1.679756,0.000000,1.507036
Yearly Mean,0.015852,1.619293,52298.000000,1.327044
Yearly Mean per Region,0.053835,1.605266,58435.571429,1.446808
Interpolate 3,-0.248698,1.837525,33301.142857,1.319927
Interpolate all,-0.249796,1.808529,59.142857,1.291759
ICE 1,0.988829,0.157992,0.000000,10.254815
ICE 2,0.956405,0.306609,0.000000,24.391101
ICE 3,0.824507,0.660810,0.000000,91.371202
MICE,0.991169,0.143301,0.000000,148.374906


In [12]:
r2_detail = pd.DataFrame(pd.DataFrame(r2_detail).T.stack().to_dict()).T
r2_detail.columns = ['r2', 'rmse', 'still_missing', 't']
r2_detail

r2      rmse  still_missing          t
Backfill 0 -0.762236  1.910158        33252.0   1.252186
         1 -0.133268  1.573266        33280.0   1.244173
         2 -0.154714  2.347292        33292.0   1.245007
         3 -0.387625  2.123361        33312.0   1.237365
         4 -0.126380  1.867463        33191.0   1.245502
...              ...       ...            ...        ...
KNN 2    2  0.469551  1.563601            0.0  19.485605
         3  0.373380  1.402108            0.0  19.581395
         4  0.125330  1.619597            0.0  19.490386
         5  0.080001  1.668049            0.0  19.675562
         6  0.241886  1.013241            0.0  19.355598

[84 rows x 4 columns]

In [13]:
r3, r3_detail = run_all(0.1)

Testdaten mit 10.0% fehlenden Werten (absolut: 17030)
Testdaten mit 10.0% fehlenden Werten (absolut: 17030)
Testdaten mit 10.0% fehlenden Werten (absolut: 17030)
Testdaten mit 10.0% fehlenden Werten (absolut: 17030)
Testdaten mit 10.0% fehlenden Werten (absolut: 17030)
Testdaten mit 10.0% fehlenden Werten (absolut: 17030)
Testdaten mit 10.0% fehlenden Werten (absolut: 17030)
Backfill
run: 1/7
r2: -3.964219549140708, rmse: 1.1521878884113992, t: 1.6051125526428223

run: 2/7
r2: -1.8373707535864638, rmse: 1.6792466658151293, t: 1.5990629196166992

run: 3/7
r2: -1.9081389310564236, rmse: 1.5239926108614468, t: 1.5961840152740479

run: 4/7
r2: -0.8977876217879388, rmse: 1.5383638773729555, t: 1.6145422458648682

run: 5/7
r2: -1.1760918948068961, rmse: 1.4134849682042556, t: 1.6395134925842285

run: 6/7
r2: -6.042109199979188, rmse: 1.7577308194911425, t: 1.5969390869140625

run: 7/7
r2: -0.965017536607846, rmse: 1.5948064365763108, t: 1.6045513153076172

[[-3.964219549140708, 1.15218788841

C:\Users\schwa\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


r2: 0.9370202334659707, rmse: 0.12777336992410396, t: 10.586702346801758

run: 2/7


C:\Users\schwa\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


r2: 0.9521427024799756, rmse: 0.2140660895293748, t: 10.648990392684937

run: 3/7


...

[IterativeImputer] Ending imputation round 8/10, elapsed time 9.26
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.41
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.58
Imputation round 7
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.14
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.29
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.44
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.60
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.73
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.91
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.06
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.22
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.36
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.53
Imputation round 8
[IterativeImputer] Comple

[IterativeImputer] Ending imputation round 6/10, elapsed time 7.08
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.24
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.38
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.55
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.69
Imputation round 6
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.15
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.28
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.46
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.62
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.78
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.92
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.10
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.26
[IterativeImputer] Ending imputation round 9/10

[IterativeImputer] Ending imputation round 4/10, elapsed time 4.63
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.77
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.92
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.05
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.19
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.35
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.49
Imputation round 5
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.15
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.32
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.45
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.60
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.75
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.90
[IterativeImputer] Ending imputation round 7/10

[IterativeImputer] Ending imputation round 2/10, elapsed time 2.29
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.42
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.60
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.74
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.89
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.04
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.22
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.36
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.59
Imputation round 4
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.14
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.31
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.46
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.62
[IterativeImputer] Ending imputation round 5/10

r2: 0.2955192171799377, rmse: 0.42734084191608557, t: 20.19080090522766

run: 2/7
r2: 0.19846267489319835, rmse: 0.8760636601263931, t: 20.315165996551514

run: 3/7
r2: -0.010296713250978806, rmse: 0.8827533199698283, t: 20.263155221939087

run: 4/7
r2: 0.17378061493706398, rmse: 0.9979210178375953, t: 20.271790027618408

run: 5/7
r2: 0.17744053064572496, rmse: 0.8546920863246453, t: 20.35792851448059

run: 6/7
r2: 0.19856669014701844, rmse: 0.5824550843672418, t: 20.374468088150024

run: 7/7
r2: 0.16675994424132778, rmse: 1.0226470021977652, t: 20.222707509994507

[[0.2955192171799377, 0.42734084191608557, 0, 20.19080090522766], [0.19846267489319835, 0.8760636601263931, 0, 20.315165996551514], [-0.010296713250978806, 0.8827533199698283, 0, 20.263155221939087], [0.17378061493706398, 0.9979210178375953, 0, 20.271790027618408], [0.17744053064572496, 0.8546920863246453, 0, 20.35792851448059], [0.19856669014701844, 0.5824550843672418, 0, 20.374468088150024], [0.16675994424132778, 1.0226470

In [14]:
r3 = pd.DataFrame(r3).T
r3

,0,1,2,3
Backfill,-2.398676,1.522830,33689.285714,1.607987
Overall Mean,-0.395046,1.014262,0.000000,1.862622
Yearly Mean,0.006562,0.877928,52320.571429,1.690804
Yearly Mean per Region,-0.047670,0.899047,58597.428571,1.799179
Interpolate 3,-0.891135,1.183015,33720.000000,1.684518
Interpolate all,-0.892295,1.163724,58.428571,1.654335
ICE 1,0.959140,0.166005,0.000000,10.599425
ICE 2,0.958234,0.169590,0.000000,24.853248
ICE 3,0.759685,0.427929,0.000000,86.695862
MICE,0.969067,0.143948,0.000000,140.654344


In [15]:
r3_detail = pd.DataFrame(pd.DataFrame(r3_detail).T.stack().to_dict()).T
r3_detail.columns = ['r2', 'rmse', 'still_missing', 't']
r3_detail

r2      rmse  still_missing          t
Backfill 0 -3.964220  1.152188        33619.0   1.605113
         1 -1.837371  1.679247        33786.0   1.599063
         2 -1.908139  1.523993        33654.0   1.596184
         3 -0.897788  1.538364        33643.0   1.614542
         4 -1.176092  1.413485        33679.0   1.639513
...              ...       ...            ...        ...
KNN 2    2 -0.010297  0.882753            0.0  20.263155
         3  0.173781  0.997921            0.0  20.271790
         4  0.177441  0.854692            0.0  20.357929
         5  0.198567  0.582455            0.0  20.374468
         6  0.166760  1.022647            0.0  20.222708

[84 rows x 4 columns]

In [16]:
t1 = time.time()
t = t1-t0
print(f'comlete runtime: {t/60}min')

comlete runtime: 164.17220553557078min


In [17]:
sets = {'r1':r1, 'r1_detail':r1_detail,
       'r2':r2, 'r2_detail':r2_detail,
       'r3':r3, 'r3_detail':r3_detail,}
for name, df in sets.items():
    df.to_csv(f'additional_data/results/{name}.csv')